In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout, GlobalAveragePooling1D, SimpleRNN, GRU, LSTM, Reshape
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.callbacks import EarlyStopping

# Load and preprocess the data
X_train = np.loadtxt("../../../../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_train.txt")
y_train = np.loadtxt("../../../../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_train.txt")
X_test = np.loadtxt("../../../../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/X_test.txt")
y_test = np.loadtxt("../../../../Datasets/CAPP Dataset/SubjectIndependent50PercentOverlap/y_test.txt")

n_timesteps = 100
n_features = 9
X_train = X_train.reshape(X_train.shape[0], n_timesteps, n_features)
X_test = X_test.reshape(X_test.shape[0], n_timesteps, n_features)

lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

n_outputs = y_train.shape[1]

# Define model architectures
def create_1d_cnn(n_timesteps, n_features, n_outputs):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        GlobalAveragePooling1D(),
        Dense(50, activation='relu'),
        Dense(n_outputs, activation='softmax')
    ])
    return model

def create_simple_rnn(n_timesteps, n_features, n_outputs):
    model = Sequential([
        SimpleRNN(50, return_sequences=True, input_shape=(n_timesteps, n_features)),
        SimpleRNN(50),
        Dense(30, activation='relu'),
        Dense(n_outputs, activation='softmax')
    ])
    return model

def create_gru(n_timesteps, n_features, n_outputs):
    model = Sequential([
        GRU(50, return_sequences=True, input_shape=(n_timesteps, n_features)),
        GRU(50),
        Dense(30, activation='relu'),
        Dense(n_outputs, activation='softmax')
    ])
    return model

def create_lightweight_lstm(n_timesteps, n_features, n_outputs):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(n_timesteps, n_features)),
        LSTM(50),
        Dense(30, activation='relu'),
        Dense(n_outputs, activation='softmax')
    ])
    return model

def create_cnn_gru(n_timesteps, n_features, n_outputs):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        GRU(50, return_sequences=True),
        GRU(50),
        Dense(n_outputs, activation='softmax')
    ])
    return model

# Function to train and evaluate a model
def train_and_evaluate(model, model_name):
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=2048,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    print(f"\n{model_name} Accuracy: {accuracy:.5f}")
    print("\nClassification Report:")
    print(classification_report(y_true_classes, y_pred_classes, digits=5))

    return model, accuracy

# Train and evaluate models
models = [
    ("1D CNN", create_1d_cnn(n_timesteps, n_features, n_outputs)),
    ("Simple RNN", create_simple_rnn(n_timesteps, n_features, n_outputs)),
    ("GRU", create_gru(n_timesteps, n_features, n_outputs)),
    ("Lightweight LSTM", create_lightweight_lstm(n_timesteps, n_features, n_outputs)),
    ("1D CNN + GRU", create_cnn_gru(n_timesteps, n_features, n_outputs))
]

results = []

for model_name, model in models:
    print(f"\nTraining {model_name}...")
    trained_model, accuracy = train_and_evaluate(model, model_name)
    results.append((model_name, trained_model, accuracy))

# Print summary of results
print("\nModel Performance Summary:")
for model_name, _, accuracy in results:
    print(f"{model_name}: {accuracy:.5f}")

# Identify the best performing model
best_model = max(results, key=lambda x: x[2])
print(f"\nBest performing model: {best_model[0]} with accuracy {best_model[2]:.5f}")

# Save the best model
best_model[1].save(f'best_model_{best_model[0].replace(" ", "_").lower()}.h5')
print(f"Best model saved as 'best_model_{best_model[0].replace(' ', '_').lower()}.h5'")

2024-09-26 16:48:39.268561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 16:48:39.279469: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 16:48:39.282728: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 16:48:39.291768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-26 16:48:39.850951: W tensorflow/compiler/tf2


Training 1D CNN...
Epoch 1/100


I0000 00:00:1727347743.636075  339845 service.cc:146] XLA service 0xaa685c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727347743.636111  339845 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-09-26 16:49:03.657735: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-26 16:49:03.748836: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-09-26 16:49:04.077207: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_483', 20 bytes spill stores, 20 bytes spill loads

2024-09-26 16:49:04.583300: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'ge

26/39 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1281 - loss: 2.9521

I0000 00:00:1727347747.736221  339845 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.4333 - loss: 1.8246 - val_accuracy: 0.5399 - val_loss: 1.5199
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.6390 - loss: 1.1998 - val_accuracy: 0.6215 - val_loss: 1.2465
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.7341 - loss: 0.8649 - val_accuracy: 0.6571 - val_loss: 1.1269
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.7792 - loss: 0.6959 - val_accuracy: 0.6775 - val_loss: 1.0897
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.8061 - loss: 0.6035 - val_accuracy: 0.6883 - val_loss: 1.0457
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.8283 - loss: 0.5380 - val_accuracy: 0.6965 - val_loss: 1.0431
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8450 - loss: 0.4875 - val_accuracy: 0.6863 - val_loss: 1.0813
Epoch 9/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8607 - loss: 0.4435 - val_accuracy: 0.7109 - val_l

W0000 00:00:1727347989.039544  339837 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347989.040111  339837 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347989.040620  339837 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347989.041126  339837 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347989.041619  339837 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347989.042120  339837 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347989.042600  339837 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347989.043104  339837 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347989.043594  339837 gp

616/616 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


W0000 00:00:1727347991.164830  339842 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347991.165421  339842 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347991.165935  339842 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347991.166443  339842 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347991.166936  339842 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347991.167437  339842 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347991.167914  339842 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347991.168424  339842 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727347991.168916  339842 gp


1D CNN + GRU Accuracy: 0.69436

Classification Report:
              precision    recall  f1-score   support

           0    0.88927   0.84898   0.86865       927
           1    0.94850   0.95361   0.95105       927
           2    0.79114   0.81169   0.80128       924
           3    0.97069   0.64196   0.77282       877
           4    0.75606   0.52748   0.62142       946
           5    0.59202   0.64681   0.61820       940
           6    0.74883   0.84958   0.79603       944
           7    0.63773   0.75789   0.69264       950
           8    0.82505   0.40168   0.54031       951
           9    0.60245   0.21251   0.31419       927
          10    0.88051   0.68003   0.76739      1322
          11    0.53635   0.90248   0.67284       564
          12    0.87155   0.87996   0.87573       933
          13    0.53711   0.98394   0.69490       934
          14    0.66520   0.64468   0.65478       940
          15    0.51922   0.74246   0.61109       928
          16    0.95129  